In [3]:
import stackexchange

api_key = '5*wCDZgONnIJ*aiaYeKjQQ(('

so = stackexchange.Site(stackexchange.StackOverflow,api_key)
so.be_inclusive()
so.impose_throttling = True
so.throttle_stop = False

from bs4 import BeautifulSoup
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk import sent_tokenize

url = 'http://stackoverflow.com/questions/3501382/checking-whether-a-variable-is-an-integer-or-not'
q_id = [3501382]
question = so.question(q_id)

p_stemmer = PorterStemmer()
tokenizer = RegexpTokenizer('\w+')

In [73]:
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import text_to_word_sequence, Tokenizer
from keras.layers import Dense, Input, GRU, Embedding
from keras.models import Model
import six.moves.cPickle as cPickle

MAX_SEQUENCE_LENGTH = 200
EMBEDDING_DIM = 300
word_index_length = 374000

embedding_layer = Embedding(word_index_length + 1,
                            EMBEDDING_DIM,
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = GRU(128, dropout_W=0.2, dropout_U=0.2)(embedded_sequences)
preds = Dense(2, activation='softmax')(x)

mymodel = Model(sequence_input, preds)
mymodel.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['acc'])
mymodel.load_weights('../Data_and_Models/stackex_gru.h5')
print('Gimme that overflow!')
vectorizer = cPickle.load(open('../Data_and_Models/rnn_tokenizer.pkl', 'rb'), encoding='latin1')
lg = mymodel

Gimme that overflow!


In [131]:
import numpy as np

def fetch_and_clean(text,model_prep=False):
    raw = BeautifulSoup(text, "lxml").get_text()
    raw = raw.lower()

    if model_prep == False:
        tokens = text_to_word_sequence(raw)
    elif model_prep == True:
        tokens = tokenizer.tokenize(raw)
        tokens = [word for word in tokens if not word.isdigit()]
        tokens = [word for word in tokens if word not in stopwords.words('english')]
        tokens = [p_stemmer.stem(i) for i in tokens]
    return tokens

def sentences_tokenization(text):
    raw = BeautifulSoup(text, "lxml").get_text()
    raw = raw.lower()
    sentences = sent_tokenize(raw)
    return sentences

def sentences_or_not(text, sentences=True):
    if sentences == False:
        output = fetch_and_clean(text)
    else:
        output = sentences_tokenization(text)
    return output

def get_answer_Info(question):
    answers_dict = {}
    text_source = question.json

    if 'comments' in text_source.keys():
        answer_id = -1
        for text in text_source['comments']:
            answer_id += 1
            temp_dict = {}
            temp_dict['tokens'] = sentences_or_not(text['body'])
            temp_dict['raw'] = text['body']
            temp_dict['score'] = text['score']
            answers_dict[answer_id] = temp_dict
    else:
        answer_id = -1

    if 'answers' in text_source.keys():
        for text in text_source['answers']:
            answer_id += 1
            temp_dict = {}
            temp_dict['tokens'] = sentences_or_not(text['body'])
            temp_dict['raw'] = [text['body']]
            temp_dict['score'] = text['score']

            if 'comments' in text.keys():
                for comments in text['comments']:
                    temp_dict['tokens'].extend(sentences_or_not(comments['body']))
                    temp_dict['raw'].extend(comments['body']) #was extend
                    temp_dict['score'] += comments['score']

            temp_dict['raw'] = ''.join(temp_dict['raw'])
            answers_dict[answer_id] = temp_dict

    return answers_dict
    
def find_helpful_sentences(answers_dict, help_threshold=0.0,print_on=False):
    new_answer_dict = answers_dict
    answers = []
    sent = []

    for keys in new_answer_dict:
        a_dict = new_answer_dict[keys]
        a_vect = a_dict['tokens']
        a_raw = a_dict['raw']
        just_tok = [fetch_and_clean(x,model_prep=False) for x in a_vect]

        #print(just_tok)
        just_tok = [vectorizer.texts_to_sequences(x) for x in just_tok]
        temp = []
        for items in just_tok: temp.append([x[0] for x in items if len(x)>0])
        just_tok = temp

        padded_seq = pad_sequences(just_tok,maxlen=MAX_SEQUENCE_LENGTH)
        seq = np.array(padded_seq)
        
        y_pred = lg.predict(seq)
        y_pred = y_pred[:,1] - y_pred[:,0]
        
        sent.append(a_vect)
        answers.append(y_pred)

    return (sent, answers)

In [132]:
answers_dict = get_answer_Info(question)
(sent, answers) = find_helpful_sentences(answers_dict)

In [133]:
from sklearn.pipeline import make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin

class MyRNN(BaseEstimator, TransformerMixin):
    def __init__(self, lg):
        self.lg = lg

    def predict(self, a_vect):
        just_tok = [fetch_and_clean(x,model_prep=False) for x in a_vect]
        just_tok = [vectorizer.texts_to_sequences(x) for x in just_tok]
        temp = []
        for items in just_tok: temp.append([x[0] for x in items if len(x)>0])
        just_tok = temp

        padded_seq = pad_sequences(just_tok,maxlen=MAX_SEQUENCE_LENGTH)
        seq = np.array(padded_seq)
        
        return self.lg.predict(seq)

In [138]:
RNN = MyRNN(lg)

array([[ 0.84269243,  0.15730758],
       [ 0.78681147,  0.2131885 ]], dtype=float32)

In [141]:
sent[1][0]

'@hulk: you seem to be under the impression that type is the right way to do this.'

In [143]:
sent_out = [item for sublist in sent for item in sublist]

In [150]:
RNN.predict([sent_out[0]])

array([[ 0.76623845,  0.23376152]], dtype=float32)

In [158]:
for i,items in enumerate(sent_out):
    if items == ':)':
        print(i)
        break

134


In [ ]:
class_names = ['unhelpful', 'helpful']
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

RNN = MyRNN(lg)
#word_dict = {}

for items in sent_out[135:]:
    exp = explainer.explain_instance(items, RNN.predict, num_features=6)
    print(items)
    print('Probability(helpful) =', RNN.predict([items])[0,1])
    word_list = exp.as_list()
    for words in word_list:
        if words[0] in word_dict.keys():
            word_dict[words[0]] = np.append(word_dict[words[0]],words[1])
        else:
            word_dict[words[0]] = words[1]

/home/dan-laptop/anaconda3/envs/insight/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


In [156]:
word_dict

{'0': 0.037732912526305631,
 '1': array([ 0.02290346,  0.05391986]),
 '12345': -0.021949733356062918,
 '21583758': 8.9702906553994894e-05,
 '3': 0.031664745299509363,
 '33': -0.017663488344971116,
 'a': array([ 0.05552529,  0.0530055 ,  0.0478736 ,  0.01688123,  0.02612986,
         0.01733033,  0.02772123,  0.02009946,  0.08322205,  0.02299505,
         0.0374692 ,  0.00979222,  0.02218186,  0.01910095,  0.04611972,
         0.01402174,  0.04203285,  0.0401535 ,  0.01301343,  0.01961553,
         0.03321212]),
 'abcs': 0.022190215250445743,
 'about': array([ 0.00148707, -0.0263881 ]),
 'above': -0.0285968200013446,
 'accordingly': -0.028738112388841081,
 'admit': 0.018820878672915757,
 'advice': -0.0082428456110958167,
 'after': array([-0.03506779, -0.04611085]),
 'agree': -0.010369007719197359,
 'ai': 0.054674352496395622,
 'all': array([-0.02677008, -0.03427115,  0.04253602,  0.03930316]),
 'almost': 0.0023442482467398579,
 'also': array([-0.0592403 , -0.08046542, -0.0858568 , -0.04

In [117]:
items

['@hulk: you seem to be under the impression that type is the right way to do this.',
 'it is (almost certainly) not.']

In [114]:
word_dict

{'canonical': 0.033773051465441559,
 'check': -0.071724063087750436,
 'duplicate': 0.039175234219797961,
 'possible': -0.024375775445865035,
 'python': 0.023019418089235359,
 's': 0.026437705602720015,
 'seem': 0.023015186501761417,
 'that': -0.0129267123790195,
 'this': -0.036796228488811485,
 'to': -0.01281247016068549,
 'way': 0.01404254601200421,
 'you': 0.056372161126580327}

## checking out what the data lime wants

In [55]:
#http://marcotcr.github.io/lime/tutorials/Lime%20-%20basic%20usage%2C%20two%20class%20case.html
from sklearn.datasets import fetch_20newsgroups
categories = ['alt.atheism', 'soc.religion.christian']
newsgroups_train = fetch_20newsgroups(subset='train', categories=categories)
newsgroups_test = fetch_20newsgroups(subset='test', categories=categories)
class_names = ['atheism', 'christian']
newsgroups_test.data[83]

'From: johnchad@triton.unm.edu (jchadwic)\nSubject: Another request for Darwin Fish\nOrganization: University of New Mexico, Albuquerque\nLines: 11\nNNTP-Posting-Host: triton.unm.edu\n\nHello Gang,\n\nThere have been some notes recently asking where to obtain the DARWIN fish.\nThis is the same question I have and I have not seen an answer on the\nnet. If anyone has a contact please post on the net or email me.\n\nThanks,\n\njohn chadwick\njohnchad@triton.unm.edu\nor\n'

In [71]:
import lime
import sklearn
import numpy as np
import sklearn
import sklearn.ensemble
import sklearn.metrics
from __future__ import print_function

vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(lowercase=False)
train_vectors = vectorizer.fit_transform(newsgroups_train.data)
test_vectors = vectorizer.transform(newsgroups_test.data)

rf = sklearn.ensemble.RandomForestClassifier(n_estimators=500)
rf.fit(train_vectors, newsgroups_train.target)

from lime import lime_text
from sklearn.pipeline import make_pipeline
c = make_pipeline(vectorizer, rf)

print(c.predict_proba([newsgroups_test.data[0]]))

from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names=class_names)

idx = 83
exp = explainer.explain_instance(newsgroups_test.data[idx], c.predict_proba, num_features=6)
print('Document id: %d' % idx)
print('Probability(christian) =', c.predict_proba([newsgroups_test.data[idx]])[0,1])
print('True class: %s' % class_names[newsgroups_test.target[idx]])


[[ 0.29  0.71]]


/home/dan-laptop/anaconda3/envs/insight/lib/python3.5/re.py:203: FutureWarning: split() requires a non-empty pattern match.
  return _compile(pattern, flags).split(string, maxsplit)


Document id: 83
Probability(christian) = 0.446
True class: unhelpful
